In [1]:
"""
Synthetic Data Augmentation, Student NN & Rule Extraction
========================================================

Workflow:

1. **Teacher** Random Forest on the original train split.
2. **SMOTE** synthetic generation up to `TARGET_TOTAL` samples.
3. **Self‑label** synthetic data with the teacher and keep only high-confidence
   points (≥ `CONF_THRESH`).
4. Merge original + synthetic data → train **MLP student**.
5. Evaluate the student on the untouched test split.
6. **DEXiRE** rule extraction from the trained student for symbolic
   interpretability, with a prettified mapping of feature indices to names.
"""

'\nSynthetic Data Augmentation, Student NN & Rule Extraction\n========================================================\n\nWorkflow:\n\n1. **Teacher** Random Forest on the original train split.\n2. **SMOTE** synthetic generation up to `TARGET_TOTAL` samples.\n3. **Self‑label** synthetic data with the teacher and keep only high-confidence\n   points (≥\xa0`CONF_THRESH`).\n4. Merge original + synthetic data → train **MLP student**.\n5. Evaluate the student on the untouched test split.\n6. **DEXiRE** rule extraction from the trained student for symbolic\n   interpretability, with a prettified mapping of feature indices to names.\n'

In [2]:
# ----------------------------------------------------------------------
# Parameters & reproducibility
# ----------------------------------------------------------------------
from __future__ import annotations

import os
import random
from pathlib import Path
import re

import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras import callbacks, layers, models, optimizers
from tensorflow.keras.utils import to_categorical

from dexire.dexire import DEXiRE  # pip install dexire
import os
from pathlib import Path
import warnings

SEED           = 42
CONF_THRESH    = 0.90       # teacher confidence threshold
TEST_SIZE      = 0.20
TARGET_TOTAL   = 10_000_000  # reduce to 2M for demo; adjust as needed
DATA_PATH      = Path(os.getenv(
    "DATASET_PATH",
    "../../data/processed_data/case_3.csv",
))
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)

In [3]:
# ----------------------------------------------------------------------
# 1⃣  Load dataset & split
# ----------------------------------------------------------------------
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Dataset not found → {DATA_PATH.resolve()}")

print("[INFO] Loading dataset …")
df = pd.read_csv(DATA_PATH)
X = df.drop(columns=["class"]).values
feature_names = df.drop(columns=["class"]).columns  # keep for DEXiRE

y = df["class"].values
le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=TEST_SIZE, stratify=y_enc, random_state=SEED
)
print(f"[INFO] Original train distribution: {np.bincount(y_train)}\n")

[INFO] Loading dataset …
[INFO] Original train distribution: [5074 5722]



In [4]:
# ----------------------------------------------------------------------
# 2⃣  Train teacher Random Forest
# ----------------------------------------------------------------------
print("[INFO] Training RandomForest teacher …")
rf = RandomForestClassifier(
    n_estimators=800,
    class_weight="balanced",
    n_jobs=-1,
    random_state=SEED,
)
rf.fit(X_train, y_train)
print("[INFO] Teacher trained.\n")

[INFO] Training RandomForest teacher …
[INFO] Teacher trained.



In [5]:
# ----------------------------------------------------------------------
# 3⃣  SMOTE synthetic generation & filtering
# ----------------------------------------------------------------------
classes           = np.unique(y_train)
per_class_target  = TARGET_TOTAL // len(classes)
smote_strategy    = {cls: per_class_target for cls in classes}
print(f"[INFO] SMOTE target per class: {smote_strategy}")

smote  = SMOTE(sampling_strategy=smote_strategy, random_state=SEED)
print("[INFO] Generating synthetic samples …")
X_syn_full, _ = smote.fit_resample(X_train, y_train)
synthetic_count = X_syn_full.shape[0] - X_train.shape[0]
X_syn = X_syn_full[-synthetic_count:]

print("[INFO] Teacher self‑labelling & confidence filter …")
proba_syn = rf.predict_proba(X_syn)
conf_max  = proba_syn.max(axis=1)
y_syn_lbl = proba_syn.argmax(axis=1)

mask_conf   = conf_max >= CONF_THRESH
X_syn_filt  = X_syn[mask_conf]
y_syn_filt  = y_syn_lbl[mask_conf]
print(f"[INFO] High‑confidence synthetic kept: {len(y_syn_filt)} / {synthetic_count}\n")

[INFO] SMOTE target per class: {0: 5000000, 1: 5000000}
[INFO] Generating synthetic samples …
[INFO] Teacher self‑labelling & confidence filter …
[INFO] High‑confidence synthetic kept: 5946459 / 9989204



In [6]:
# ----------------------------------------------------------------------
# 4⃣  Merge & shuffle
# ----------------------------------------------------------------------
X_comb = np.vstack([X_train, X_syn_filt])
y_comb = np.concatenate([y_train, y_syn_filt])
perm    = np.random.permutation(len(y_comb))
X_comb, y_comb = X_comb[perm], y_comb[perm]
print(f"[INFO] Final train size: {X_comb.shape}, distribution: {np.bincount(y_comb)}\n")


[INFO] Final train size: (5957255, 21), distribution: [2685235 3272020]



In [7]:
# ----------------------------------------------------------------------
# 5⃣  Scale & one‑hot labels
# ----------------------------------------------------------------------
scaler = StandardScaler()
X_comb_s = scaler.fit_transform(X_comb)
X_test_s = scaler.transform(X_test)

y_comb_o = to_categorical(y_comb)
y_test_o = to_categorical(y_test)

In [8]:
# ----------------------------------------------------------------------
# 6⃣  Build + train student MLP
# ----------------------------------------------------------------------
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Assumes X_train, y_train, X_val, y_val, X_test, y_test are already prepared
# and class_weight is defined if you want to balance classes

# 1. Costruzione del modello



def build_student(input_dim: int, output_dim: int):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        # Dropout(0.3),
        Dense(64, activation='relu'),
        # Dropout(0.3),
        Dense(32, activation='relu'),
        # Dropout(0.3),
        Dense(output_dim, activation='sigmoid')
    ])
    model.compile(optimizer=optimizers.Adam(1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

print("[INFO] Training student MLP …")
student = build_student(X_comb_s.shape[1], y_comb_o.shape[1])
cb_es = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
cb_lr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5)
student.fit(
    X_comb_s,
    y_comb_o,
    validation_split=0.2,
    epochs=30,
    batch_size=1024,
    callbacks=[cb_es, cb_lr],
    verbose=1,
)

[INFO] Training student MLP …
Epoch 1/30
4655/4655 [==============================] - 6s 1ms/step - loss: 0.0623 - accuracy: 0.9817 - val_loss: 0.0367 - val_accuracy: 0.9871 - lr: 0.0010
Epoch 2/30
4655/4655 [==============================] - 6s 1ms/step - loss: 0.0342 - accuracy: 0.9885 - val_loss: 0.0384 - val_accuracy: 0.9867 - lr: 0.0010
Epoch 3/30
4655/4655 [==============================] - 6s 1ms/step - loss: 0.0265 - accuracy: 0.9908 - val_loss: 0.0214 - val_accuracy: 0.9924 - lr: 0.0010
Epoch 4/30
4655/4655 [==============================] - 6s 1ms/step - loss: 0.0227 - accuracy: 0.9924 - val_loss: 0.0179 - val_accuracy: 0.9941 - lr: 0.0010
Epoch 5/30
4655/4655 [==============================] - 6s 1ms/step - loss: 0.0198 - accuracy: 0.9934 - val_loss: 0.0178 - val_accuracy: 0.9932 - lr: 0.0010
Epoch 6/30
4655/4655 [==============================] - 6s 1ms/step - loss: 0.0188 - accuracy: 0.9939 - val_loss: 0.0163 - val_accuracy: 0.9947 - lr: 0.0010
Epoch 7/30
4655/4655 [======

In [9]:
# ----------------------------------------------------------------------
# 7⃣  Evaluation
# ----------------------------------------------------------------------
print("[INFO] Evaluating student …")
loss, acc = student.evaluate(X_test_s, y_test_o, verbose=0)
print(f"[RESULT] TEST – Loss: {loss:.4f} | Accuracy: {acc:.4f}\n")

y_pred = np.argmax(student.predict(X_test_s, verbose=0), axis=1)
print(classification_report(y_test, y_pred, target_names=le.classes_))
print(confusion_matrix(y_test, y_pred))

[INFO] Evaluating student …
[RESULT] TEST – Loss: 0.8320 | Accuracy: 0.8907

              precision    recall  f1-score   support

       fraud       0.89      0.87      0.88      1269
       licit       0.89      0.91      0.90      1430

    accuracy                           0.89      2699
   macro avg       0.89      0.89      0.89      2699
weighted avg       0.89      0.89      0.89      2699

[[1105  164]
 [ 131 1299]]


In [10]:
_ = student.predict(X_comb_s[:1])  # oppure: student(X_comb_s[:1], training=False)

# ----------------------------------------------------------------------
# 8⃣  Symbolic Rule Extraction with DEXiRE
# ----------------------------------------------------------------------
print("[INFO] Extracting rules with DEXiRE … (this may take a while)")

# Build DataFrame of scaled features for DEXiRE
features_df = pd.DataFrame(X_comb_s, columns=feature_names)

dexire = DEXiRE(model=student, class_names=le.classes_.tolist())
rules_raw = dexire.extract_rules(features_df, y_comb)

# Save raw rules
rules_path = Path("rules_student.txt")
rules_path.write_text(str(rules_raw), encoding="utf-8")
print(f"[INFO] Raw rules saved → {rules_path}")

# Prettify placeholders (X_i) with real feature names
idx2name = {i: name for i, name in enumerate(feature_names)}

def repl(match):
    return idx2name.get(int(match.group(1)), match.group(0))

rules_pretty = re.sub(r"X_(\d+)", repl, str(rules_raw))
pretty_path = Path("rules_student_pretty.txt")
pretty_path.write_text(rules_pretty, encoding="utf-8")
print(f"[INFO] Pretty rules saved → {pretty_path}")

1/1 [==============================] - 0s 8ms/step
[INFO] Extracting rules with DEXiRE … (this may take a while)
186165/186165 [==============================] - 43s 229us/step
Unique predictions: [0 1]
186165/186165 [==============================] - 39s 211us/step
73
feature_73
Symbolic expression: feature_73 <= 0.5
30
feature_30
Symbolic expression: feature_30 <= 0.5
125
feature_125
Symbolic expression: feature_125 <= 0.5
36
feature_36
Symbolic expression: feature_36 <= 0.5
57
feature_57
Symbolic expression: feature_57 <= 0.5
82
feature_82
Symbolic expression: feature_82 <= 0.5
20
feature_20
Symbolic expression: feature_20 <= 0.5
4
feature_4
Symbolic expression: feature_4 <= 0.5
9
feature_9
Symbolic expression: feature_9 <= 0.5
41
feature_41
Symbolic expression: feature_41 <= 0.5
90
feature_90
Symbolic expression: feature_90 <= 0.5
Symbolic expression: feature_90 > 0.5
Symbolic expression: feature_41 > 0.5
Symbolic expression: feature_9 > 0.5
Symbolic expression: feature_4 > 0.5
Sym